<a href="https://colab.research.google.com/github/adelhoyov/03MAIR-Algoritmos_de_Optimizacion/blob/main/Algoritmos_de_Optimizacion_Trabajo_Practico_Alejandro_del_Hoyo_Vijande.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Alejandro del Hoyo Vijande  <br>
Url: https://github.com/adelhoyov/03MAIR-Algoritmos_de_Optimizacion/tree/main/TRABAJO_PRACTICO<br>
Google Colab: https://colab.research.google.com/drive/1fX_KgIVI8CQ32KW8ztuohMw-Nw0BoWkG <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de una jornada de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:

## Problema 1. Organizar sesiones de doblaje:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:

- Número de actores: 10
- Número de tomas: 30
- Actores/Toma: https://bit.ly/36D8IuK (1 indica que el actor participa en la toma, 0 en caso contrario)








                                        

In [2]:
# Importamos las librerías que vamos a utilizar:
import pandas as pd
import numpy as np
import random

In [3]:
# Accedemos a la hoja de cálculo de Google que contiene los datos del problema:
sheet_id = '1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0'
sheet_name = 'Hoja1'

# Enlace para exportar a CSV
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Cargamos los datos en un DataFrame de Pandas:
df = pd.read_csv(url)

# Generamos una matriz de datos a partir del DataFrame:
data_matrix = df.iloc[1:-1, 1:-2].to_numpy()
total_actores_por_toma = df.iloc[1:-1, -1].to_numpy()

print('MATRIZ DE DATOS: Las filas se corresponden con cada una de las tomas que se deben grabar y las columnas representan los actores.')
print('\t- 1 indica que el actor participa en la toma.\n\t- 0 en caso contrario.\n')
print(data_matrix)

print('\nVector con el numero total de actores que participan en cada toma:', total_actores_por_toma)

MATRIZ DE DATOS: Las filas se corresponden con cada una de las tomas que se deben grabar y las columnas representan los actores.
	- 1 indica que el actor participa en la toma.
	- 0 en caso contrario.

[[1 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0]
 [0 1 0 0 1 0 1 0 0 0]
 [1 1 0 0 0 0 1 1 0 0]
 [0 1 0 1 0 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0 0]
 [1 1 0 1 1 0 0 0 0 0]
 [1 1 0 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0]
 [1 1 0 0 0 1 0 0 1 0]
 [1 1 1 0 1 0 0 1 0 0]
 [1 1 1 1 0 1 0 0 0 0]
 [1 0 0 1 1 0 0 0 0 0]
 [1 0 1 0 0 1 0 0 0 0]
 [1 1 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 1]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [1 1 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 1]
 [1 0 1 0 1 0 0 0 1 0]
 [0 0 0 1 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0]]

Vector con el numero total de actores que participan en cada toma: [5. 3. 3. 4. 3. 4. 4. 3. 3. 4. 5. 5. 3. 

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

### Representación del espacio de soluciones:
Las posibles soluciones del problema se han modelado como una lista de listas. Cada una de las listas internas (sub-listas) representará un día de grabación, y los elementos internos de cada sub-lista indican los números de las tomas que se grabarán dicho día. El número de días de grabación es flexible, pero en este problema en concreto debe ser igual o superior a 5 debido a las restricciones que se detallarán más adelante. Para que una solución sea válida no debe haber números repetidos en la lista de listas, ya que se asume que todas las tomas se terminan el mismo día que empieza su grabación.



### Función objetivo:
La función objetivo que se tratará de **minimizar** en este problema es el número total de desplazamientos de los actores de doblaje, ya que cobran por cada día que deben desplazarse hasta el estudio. Por tanto, para calcular el coste de una solución se sumará el número de actores que deben desplazarse hasta el estudio cada uno de los días ($A_d$) en base a la programación de las tomas. Cabe destacar que, dada la naturaleza del problema, podría darse el caso de que una solución con un mayor número de días ($D$) tuviese un coste inferior que otra con un total de días inferior. Por esta razón, se adoptó un enfoque flexible en el modelado de las soluciones.

$$
\text{Coste de una solución} = \sum_{d=1}^{D} |A_d|
$$

### Implementación de las restricciones:
Como se especifica en el enunciado del problema, no será posible grabar más de 6 tomas diarias. Para implementar esta restricción, las sub-listas no tendrán más de 6 elementos (por eso el número mínimo de días para grabar las 30 tomas es 5). Aunque no se especifique de forma implícita en el enunciado, todas las tomas deben ser asignadas y no deben repetirse. Para asegurar que estas nuevas restricciones se cumplen, el punto de partida para generar soluciones será una lista desordenada con todos los números naturales del 1 al 30, que posteriormente se agrupará en sub-listas con un máximo de 6 elementos.

In [4]:
# Ejemplo de una posible solución del problema:
lista = [[6, 13, 26, 30],[1, 2, 5, 14],[4, 18, 25],[3, 11, 12, 19],[8, 16, 20, 27],[9, 10, 21, 22, 23, 28],[7, 15, 17, 24, 29]]
display(lista)

[[6, 13, 26, 30],
 [1, 2, 5, 14],
 [4, 18, 25],
 [3, 11, 12, 19],
 [8, 16, 20, 27],
 [9, 10, 21, 22, 23, 28],
 [7, 15, 17, 24, 29]]

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Nos encontramos ante un problema de optimización combinatoria. Por lo general, este tipo de problemas son de la clase *NP-hard*, de tal forma que no será posible explorar la totalidad del espacio de soluciones para encontrar la óptima a partir de una cierta dimensión (i.e., número de tomas en este caso).  El orden de complejidad es exponencial: O($k^n$). Suponiendo que solo se contemplasen soluciones de 5 días para grabar las 30 tomas, el espacio de soluciones sería $5^{30} \approx 9.31·10^{20}$, lo cual resulta inmanejable siguiendo enfoques deterministas. Si bien es cierto que la solución óptima no será alcanzable en instancias del problema como la que aquí se plantea, utilizar algoritmos heurísticos permitirá obtener soluciones consideradas como buenas o aceptables.

**Referencia**: Vicentini, F., & Puddu, S. (2003). *Algoritmos heurísticos y el problema de job shop scheduling*.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Dado que el problema es *NP-hard* y el espacio de soluciones es inmenso, la mejor estrategia involucra el uso de metaheurísticas. En el enfoque que se propone en este trabajo se emplearon dos técnicas basadas en trayectorias: Simulated Annealing y Búsqueda Tabú. La primera de ellas se puede considerar mejor para escapar de mínimos locales, ya que contempla la posibilidad de cambiar de trayectoria en función de la temperatura en una determinada iteración y la diferencia entre soluciones de iteraciones consecutivas. La Búsqueda Tabú, por otra parte, es mejor en términos de intensificación o explotación, ya que se mueve siempre en un entorno local evitando ciclos de soluciones repetidas (gracias a la lista tabú). Además, con objeto de mejorar la capacidad de diversificación o exploración de los algoritmos, se propone un enfoque multiarranque en el que se parte de soluciones aleatorias diferentes sobre las que se aplican los algoritmos de optimización. Tras realizar diferentes pruebas con distintos parámetros, la solución basada en Simulated Annealing ha dado mejores resultados que la basada en Búsqueda Tabú.

In [5]:

# Generación de las soluciones iniciales:
# (flexible en cuanto al número de tomas hasta el máximo permitido de 6 diarias)
def generar_solucion_inicial(num_tomas, max_dias):
    tomas = list(range(1, num_tomas + 1))
    random.shuffle(tomas) # Mezclamos la lista para generar una solución aleatoria

    # Definimos un número de días entre 5 y max_dias:
    num_dias = random.randint(5, max_dias)

    # Crear la solución inicial dividiendo las tomas en num_dias días
    solucion = [[] for i in range(num_dias)] # Modelado de la solución: Lista de listas
    for j, toma in enumerate(tomas):
        solucion[j % num_dias].append(toma)

    return solucion


# Función de coste:
def calcular_coste(solucion, matriz_participacion):
    coste_total = 0
    for dia_idx, dia in enumerate(solucion):
        actores_presentes = set() # Conjunto de actores que participan ese día
        for toma in dia:
            actores_en_toma = set(np.where(matriz_participacion[toma-1] == 1)[0])
            actores_presentes.update(actores_en_toma)
        coste_dia = len(actores_presentes)
        actores_presentes = {a + 1 for a in actores_presentes}
        coste_total += coste_dia # El coste de una solución será el total de desplazamientos para grabar todas las tomas
    return coste_total


# Función para mostrar por consola una solución de forma detallada:
def presentar_solucion(solucion, matriz_participacion):
    coste_total = 0
    for idx_dia, tomas_dia in enumerate(solucion):
        actores_presentes = set() # Conjunto de actores que participan ese día
        for toma in tomas_dia:
            actores_en_toma = set(np.where(matriz_participacion[toma-1] == 1)[0])
            actores_presentes.update(actores_en_toma)
        coste_dia = len(actores_presentes)
        actores_presentes = {a + 1 for a in actores_presentes}
        print(f"El día {idx_dia + 1} se graban las siguientes tomas: {tomas_dia} -> Actores: {actores_presentes}, Coste: {coste_dia}")
        coste_total += coste_dia
    print(f"Coste total: {coste_total} desplazamientos a pagar.\n")


# Generación de vecinos:
def generar_vecino(solucion):
    nueva_solucion = [list(dia) for dia in solucion]

    # Selección aleatoria de dos días diferentes:
    dia1, dia2 = random.sample(range(len(nueva_solucion)), 2)

    if nueva_solucion[dia1] and nueva_solucion[dia2]:  # (Ambos días deben tener al menos una toma)
        toma1 = random.choice(nueva_solucion[dia1])
        toma2 = random.choice(nueva_solucion[dia2])

        # Intercambio de tomas entre los días seleccionados aleatoriamente:
        nueva_solucion[dia1].remove(toma1)
        nueva_solucion[dia1].append(toma2)
        nueva_solucion[dia2].remove(toma2)
        nueva_solucion[dia2].append(toma1)

    return nueva_solucion


# Implementación de Simulated Annealing para la planificación del doblaje:
def simulated_annealing(tabla_participacion, proceso, max_dias = 6, temp_inicial = 1000, temp_final = 1, alpha = 0.99, iter_por_temp = 100):
    '''
        PARÁMETROS DE ENTRADA DE LA FUNCIÓN:
            tabla_participacion ---> Matriz que relaciona actores y tomas.
            proceso ---------------> Índice del proceso (enfoque multiarranque).
            max_dias --------------> Máximo de días para delimitar el espacio de soluciones
                                    (el mínimo son 5 días, a razón de 6 tomas diarias).
            temp_inicial ----------> Temperatura inicial (valor alto).
            temp_final ------------> Temperatura final (condición de parada).
            alpha -----------------> Factor de reducción de la temperatura.
            iter_por_temp ---------> Número de iteraciones para cada valor de temperatura.

        La función retorna la MEJOR SOLUCIÓN ENCONTRADA y su COSTE ASOCIADO.
    '''
    num_tomas = len(tabla_participacion) # Número de tomas.

    solucion_actual = generar_solucion_inicial(num_tomas, max_dias)
    mejor_solucion = solucion_actual
    mejor_coste = calcular_coste(solucion_actual, tabla_participacion)
    temperatura = temp_inicial

    while temperatura > temp_final:
        for _ in range(iter_por_temp):
            vecino = generar_vecino(solucion_actual)
            coste_vecino = calcular_coste(vecino, tabla_participacion)
            delta = coste_vecino - mejor_coste

            if delta < 0 or random.uniform(0, 1) < np.exp(-delta / temperatura):
                solucion_actual = vecino
                mejor_coste = coste_vecino
                mejor_solucion = vecino

        temperatura *= alpha

    return mejor_solucion, mejor_coste


# Implementación de Búsqueda Tabú para la planificación del doblaje:
def busqueda_tabu(tabla_participacion, proceso, max_dias = 6, n_vecinos = 10, iteraciones = 1000, lista_tabu_tam = 50, sin_mejora_max = 100):
    '''
        PARÁMETROS DE ENTRADA DE LA FUNCIÓN:
            tabla_participacion ---> Matriz que relaciona actores y tomas.
            proceso ---------------> Índice del proceso (enfoque multiarranque).
            max_dias --------------> Máximo de días para delimitar el espacio de soluciones
                                     (el mínimo son 5 días, a razón de 6 tomas diarias).
            n_vecinos -------------> Número de vecinos generados por iteración.
            iteraciones -----------> Número máximo de iteraciones del algoritmo.
            lista_tabu_tam --------> Tamaño máximo de la lista tabú.
            sin_mejora_max --------> Numero máximo de iteraciones sin mejora
                                     (condición de parada anticipada).

        La función retorna la MEJOR SOLUCIÓN ENCONTRADA y su COSTE ASOCIADO.
    '''
    solucion_actual = generar_solucion_inicial(num_tomas = len(tabla_participacion), max_dias = max_dias)
    mejor_solucion = solucion_actual
    mejor_coste = calcular_coste(solucion_actual, tabla_participacion)

    lista_tabu = []
    sin_mejora = 0  # Número de iteraciones en las que no hemos conseguido mejorar.

    for iter in range(iteraciones):
        vecinos = [generar_vecino(solucion_actual) for j in range(n_vecinos)]  # Generamos "n" vecinos
        vecinos = [v for v in vecinos if v not in lista_tabu]  # Eliminamos los movimientos tabú

        if not vecinos:
            continue  # Si no hay vecinos válidos, saltar iteración

        # Evaluamos el mejor vecino:
        vecino_mejor = min(vecinos, key = lambda v: calcular_coste(v, tabla_participacion))
        coste_vecino = calcular_coste(vecino_mejor, tabla_participacion)
        # print(f"Proceso {proceso}: Coste total de {coste_vecino} desplazamientos en la solución de la iteración {iter}.")

        # Actualizamos la mejor solución (si es necesario):
        if coste_vecino < mejor_coste:
            mejor_solucion = vecino_mejor
            mejor_coste = coste_vecino
            sin_mejora = 0
        else:
            sin_mejora += 1

        # Añadimos el mejor vecino a la lista tabú:
        lista_tabu.append(vecino_mejor)
        if len(lista_tabu) > lista_tabu_tam: # Para no exceder el límite de la lista.
            lista_tabu.pop(0)

        # Condición de parada anticipada si no conseguimos mejorar
        # durante "sin_mejora_max" iteraciones:
        if sin_mejora >= sin_mejora_max:
            break

    return mejor_solucion, mejor_coste


# Función para ejecutar múltiples arranques de los algoritmos de optimización:
def multiarranque(algoritmo, tabla_participacion, n_runs = 5, **params):
    resultados = {}  # Diccionario para almacenar soluciones y costes.

    for i in range(n_runs):
        random.seed(i)  # Diferentes semillas para diversidad de soluciones
        np.random.seed(i)

        if algoritmo == "SA": # Simulated Annealing
            solucion, coste = simulated_annealing(tabla_participacion, proceso=i, **params)
        elif algoritmo == "BT": # Búsqueda Tabú
            solucion, coste = busqueda_tabu(tabla_participacion, proceso=i, **params)
        else:
            raise ValueError("Algoritmo no reconocido. Usa 'SA' o 'BT'.")

        # Guardamos la solución y el coste en el diccionario:
        resultados[i] = {"solucion": solucion, "coste": coste}
        print(f"Arranque {i}: Coste = {coste}")

    # Extraemos la mejor solución encontrada:
    mejor_proceso = min(resultados, key=lambda k: resultados[k]["coste"])
    mejor_solucion = resultados[mejor_proceso]["solucion"]
    mejor_coste = resultados[mejor_proceso]["coste"]

    print(f"\nMejor solución encontrada en el arranque {mejor_proceso} con coste {mejor_coste}.\n")

    return mejor_solucion, mejor_coste, resultados


In [8]:
# Prueba de generación aleatoria de soluciones y cálculo de coste:
n_tomas = 30
max_dias = 8

sol = generar_solucion_inicial(n_tomas, max_dias)
print("Solución generada de forma aleatoria:")
display(sol)

coste = calcular_coste(sol, data_matrix)
print(f"Coste de la solución: {coste} desplazamientos.")

Solución generada de forma aleatoria:


[[30, 10, 4, 17],
 [22, 23, 19, 1],
 [12, 26, 27, 18],
 [11, 29, 8, 24],
 [25, 28, 16, 7],
 [2, 15, 13, 5],
 [9, 21, 20],
 [6, 3, 14]]

Coste de la solución: 52 desplazamientos.


In [9]:
# Obtención de una única colución ejecutando el algoritmo de Simulated Annealing (SA):
sol_SA, coste_SA = simulated_annealing(data_matrix, proceso = 0, max_dias = 6, temp_inicial = 2000, temp_final = 0.1, alpha = 0.99, iter_por_temp = 120)
presentar_solucion(sol_SA, data_matrix)

El día 1 se graban las siguientes tomas: [4, 15, 3, 29, 8, 21] -> Actores: {1, 2, 5, 6, 7, 8}, Coste: 6
El día 2 se graban las siguientes tomas: [12, 10, 26, 22, 2, 1] -> Actores: {1, 2, 3, 4, 5, 6, 9}, Coste: 7
El día 3 se graban las siguientes tomas: [16, 25, 30, 28, 9, 5] -> Actores: {1, 2, 4, 8, 10}, Coste: 5
El día 4 se graban las siguientes tomas: [13, 11, 7, 27, 20, 6] -> Actores: {1, 2, 3, 4, 5, 8}, Coste: 6
El día 5 se graban las siguientes tomas: [18, 14, 17, 19, 23, 24] -> Actores: {1, 3, 6}, Coste: 3
Coste total: 27 desplazamientos a pagar.



In [13]:
# Obtención de una única colución ejecutando el algoritmo de Búsqueda Tabú (BT):
sol_tabu, coste_tabu = busqueda_tabu(data_matrix, proceso = 0, max_dias = 6, n_vecinos = 20, iteraciones = 2000, lista_tabu_tam = 30, sin_mejora_max = 300)
presentar_solucion(sol_tabu, data_matrix)

El día 1 se graban las siguientes tomas: [24, 9, 12, 25, 15, 8] -> Actores: {1, 2, 3, 4, 6, 7, 10}, Coste: 7
El día 2 se graban las siguientes tomas: [6, 1, 21, 22, 3, 18] -> Actores: {1, 2, 3, 4, 5, 6, 7, 8}, Coste: 8
El día 3 se graban las siguientes tomas: [23, 20, 30, 17, 28, 13] -> Actores: {1, 3, 4, 5}, Coste: 4
El día 4 se graban las siguientes tomas: [26, 29, 27, 2, 19, 16] -> Actores: {1, 3, 4, 5, 6, 9, 10}, Coste: 7
El día 5 se graban las siguientes tomas: [10, 11, 7, 14, 5, 4] -> Actores: {1, 2, 3, 4, 5, 6, 7, 8, 9}, Coste: 9
Coste total: 35 desplazamientos a pagar.



In [14]:
# Enfoque multiarranque con Simulated Annealing (SA):
mejor_sol, mejor_coste, resultados = multiarranque("SA", data_matrix, n_runs = 10, max_dias = 6, temp_inicial = 2000, temp_final = 0.1, alpha=0.99, iter_por_temp=120)

# display(resultados)
presentar_solucion(mejor_sol, data_matrix)

Arranque 0: Coste = 30
Arranque 1: Coste = 27
Arranque 2: Coste = 30
Arranque 3: Coste = 27
Arranque 4: Coste = 31
Arranque 5: Coste = 27
Arranque 6: Coste = 31
Arranque 7: Coste = 30
Arranque 8: Coste = 30
Arranque 9: Coste = 27

Mejor solución encontrada en el arranque 1 con coste 27.

El día 1 se graban las siguientes tomas: [25, 16, 27, 7, 13, 6] -> Actores: {1, 2, 4, 5, 10}, Coste: 5
El día 2 se graban las siguientes tomas: [12, 26, 10, 2, 30, 28] -> Actores: {1, 2, 3, 4, 5, 6, 9}, Coste: 7
El día 3 se graban las siguientes tomas: [3, 15, 4, 21, 29, 8] -> Actores: {1, 2, 5, 6, 7, 8}, Coste: 6
El día 4 se graban las siguientes tomas: [14, 18, 23, 24, 19, 17] -> Actores: {1, 3, 6}, Coste: 3
El día 5 se graban las siguientes tomas: [11, 5, 1, 22, 20, 9] -> Actores: {1, 2, 3, 4, 5, 8}, Coste: 6
Coste total: 27 desplazamientos a pagar.



In [17]:
# Enfoque multiarranque con Búsqueda Tabú (BT):
mejor_sol, mejor_coste, resultados = multiarranque("BT", data_matrix, n_runs = 500, max_dias = 6, n_vecinos = 20, iteraciones = 2000, lista_tabu_tam = 30, sin_mejora_max = 300)

# display(resultados)
presentar_solucion(mejor_sol, data_matrix)

Arranque 0: Coste = 41
Arranque 1: Coste = 37
Arranque 2: Coste = 42
Arranque 3: Coste = 36
Arranque 4: Coste = 42
Arranque 5: Coste = 36
Arranque 6: Coste = 41
Arranque 7: Coste = 40
Arranque 8: Coste = 39
Arranque 9: Coste = 38
Arranque 10: Coste = 42
Arranque 11: Coste = 36
Arranque 12: Coste = 41
Arranque 13: Coste = 34
Arranque 14: Coste = 37
Arranque 15: Coste = 36
Arranque 16: Coste = 41
Arranque 17: Coste = 36
Arranque 18: Coste = 39
Arranque 19: Coste = 41
Arranque 20: Coste = 36
Arranque 21: Coste = 37
Arranque 22: Coste = 37
Arranque 23: Coste = 40
Arranque 24: Coste = 37
Arranque 25: Coste = 37
Arranque 26: Coste = 36
Arranque 27: Coste = 42
Arranque 28: Coste = 36
Arranque 29: Coste = 40
Arranque 30: Coste = 36
Arranque 31: Coste = 43
Arranque 32: Coste = 42
Arranque 33: Coste = 37
Arranque 34: Coste = 40
Arranque 35: Coste = 40
Arranque 36: Coste = 37
Arranque 37: Coste = 41
Arranque 38: Coste = 37
Arranque 39: Coste = 41
Arranque 40: Coste = 36
Arranque 41: Coste = 35
Ar